# Assignment #3: A simple language classifier

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [4]:
import bz2
import json
import os
import numpy as np
import requests
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix

### Dataset

In [5]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tatoeba/sentences.csv


As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider three languages only: French (fra), English (eng), and Swedish (swe). Below is an excerpt of the Tatoeba dataset limited to these three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 15, 2020.

### Understanding the $\mathbf{X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified $\mathbf{X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will include this matrix in your report. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

## Programming: Extracting the features

Before you start programming, download the Tatoeba dataset.

### Loading and filtering the dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [6]:
dataset = open('/kaggle/input/tatoeba/sentences.csv', encoding='utf8').read().strip()
dataset = dataset.split('\n')
dataset[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

Run the code to split the fields and remove possible whitespaces

In [7]:
dataset = list(map(lambda x: tuple(x.split('\t')), dataset))
dataset = list(map(lambda x: tuple(map(str.strip, x)), dataset))
dataset[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

Write the code to extract the French, English, and Swedish texts. You will call the resulting dataset: `dataset_small`

In [8]:
dataset_small = list(filter(lambda x: x[1] == 'fra' or x[1] == 'eng' or x[1] == 'swe', dataset))
del dataset # clear some memory

In [9]:
dataset_small[:5]

[('1115',
  'fra',
  "Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."),
 ('1276', 'eng', "Let's try something."),
 ('1277', 'eng', 'I have to go to sleep.'),
 ('1279', 'fra', 'Je ne supporte pas ce type.'),
 ('1280', 'eng', "Today is June 18th and it is Muiriel's birthday!")]

### Functions to Count Characters Ngrams

Write a function `count_chars(string, lc=True)` to count characters (unigrams) of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the unigrams.

In [10]:
def count_chars(string, lc=True):
    if lc == True:
        string = string.lower()
    frequency = {}
    for char in string:
        if char in frequency:
            frequency[char] +=1 / len(string)
        else:
            frequency[char] = 1 / len(string)
    return frequency

Write a function `count_bigrams(string, lc=True)` to count the characters bigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the bigrams.

In [11]:
def count_bigrams(string, lc=True):
    if lc == True:
        string = string.lower()
    frequency = {}
    bigrams = [string[inx:inx + 2]
                   for inx in range(len(string) - 1)]
    for bg in bigrams:
        if bg in frequency:
            frequency[bg] +=1 / len(string)
        else:
            frequency[bg] = 1 / len(string)
    return frequency

Write a function `count_trigrams(string, lc=True)` to count the characters trigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the trigrams.

In [12]:
def count_trigrams(string, lc=True):
    if lc == True:
        string = string.lower()
    frequency = {}
    trigrams = [string[inx:inx + 3]
                   for inx in range(len(string) - 2)]
    for tg in trigrams:
        if tg in frequency:
            frequency[tg] +=1 / len(string)
        else:
            frequency[tg] = 1 / len(string)
    return frequency

In [13]:
count_chars("Let's try something.")

{'l': 0.05,
 'e': 0.1,
 't': 0.15000000000000002,
 "'": 0.05,
 's': 0.1,
 ' ': 0.1,
 'r': 0.05,
 'y': 0.05,
 'o': 0.05,
 'm': 0.05,
 'h': 0.05,
 'i': 0.05,
 'n': 0.05,
 'g': 0.05,
 '.': 0.05}

In [14]:
count_bigrams("Let's try something.")

{'le': 0.05,
 'et': 0.1,
 "t'": 0.05,
 "'s": 0.05,
 's ': 0.05,
 ' t': 0.05,
 'tr': 0.05,
 'ry': 0.05,
 'y ': 0.05,
 ' s': 0.05,
 'so': 0.05,
 'om': 0.05,
 'me': 0.05,
 'th': 0.05,
 'hi': 0.05,
 'in': 0.05,
 'ng': 0.05,
 'g.': 0.05}

In [15]:
count_trigrams("Let's try something.")

{'let': 0.05,
 "et'": 0.05,
 "t's": 0.05,
 "'s ": 0.05,
 's t': 0.05,
 ' tr': 0.05,
 'try': 0.05,
 'ry ': 0.05,
 'y s': 0.05,
 ' so': 0.05,
 'som': 0.05,
 'ome': 0.05,
 'met': 0.05,
 'eth': 0.05,
 'thi': 0.05,
 'hin': 0.05,
 'ing': 0.05,
 'ng.': 0.05}

### Counting the ngrams in the dataset

You will now extract the features from each text. For this, add the character, bigram, and trigram relative frequencies to the texts using this format:
`(text_id, language_id, text, char_cnt, bigram_cnt, trigram_cnt)`.

From the datapoint:
`('1276', 'eng', "Let's try something.")`,
you must return:

`('1276', 'eng', "Let's try something.", 
  {'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 'n': 0.05, 'g': 0.05, '.': 0.05},
  {'le': 0.05263157894736842, 'et': 0.10526315789473684, "t'": 0.05263157894736842, "'s": 0.05263157894736842, 's ': 0.05263157894736842, ' t': 0.05263157894736842, 'tr': 0.05263157894736842, 'ry': 0.05263157894736842, 'y ': 0.05263157894736842, ' s': 0.05263157894736842, 'so': 0.05263157894736842, 'om': 0.05263157894736842, 'me': 0.05263157894736842, 'th': 0.05263157894736842, 'hi': 0.05263157894736842, 'in': 0.05263157894736842, 'ng': 0.05263157894736842, 'g.': 0.05263157894736842},
  {'let': 0.05555555555555555, "et'": 0.05555555555555555, "t's": 0.05555555555555555, "'s ": 0.05555555555555555, 's t': 0.05555555555555555, ' tr': 0.05555555555555555, 'try': 0.05555555555555555, 'ry ': 0.05555555555555555, 'y s': 0.05555555555555555, ' so': 0.05555555555555555, 'som': 0.05555555555555555, 'ome': 0.05555555555555555, 'met': 0.05555555555555555, 'eth': 0.05555555555555555, 'thi': 0.05555555555555555, 'hin': 0.05555555555555555, 'ing': 0.05555555555555555, 'ng.': 0.05555555555555555})`

You will store the extracted features in a list that you will call `dataset_small_feat`

In [ ]:
def get_features(dataset):
    ext_dataset = []
    for data in dataset:
        ext = data + (count_chars(data[2]), count_bigrams(data[2]), count_trigrams(data[2]),)
        ext_dataset.append(ext)
    return ext_dataset
    
dataset_small_feat = get_features(dataset_small)
del dataset_small

In [ ]:
dataset_small_feat[:2]

The unigram frequencies

In [ ]:
dataset_small_feat[0][3].items()

The bigram frequencies

In [ ]:
dataset_small_feat[0][4].items()

## Programming: Building $\mathbf{X}$

You will now build the $\mathbf{X}$ matrix. In this assignment, you will only consider unigrams to speed up the training step. This means that you will set aside the character bigrams and trigrams.

When you are done with the lab requirements, feel free to improve the program and include bigrams and trigrams. To add bigrams, a possible method is to add the bigram dictionary to the unigram one using update and then to extract the resulting dictionary. You can easily extend this to trigrams. Feel free to use another method if you want.

In [ ]:
INCLUDE_BIGRAMS = False
if INCLUDE_BIGRAMS:
    for i in range(len(dataset_small_feat)):
        dataset_small_feat[i][3].update(dataset_small_feat[i][4])

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

To create the $\mathbf{X}$ matrix, we need to transform the dictionaries of `dataset_small` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the character frequency dictionaries from `dataset_small` corresponding to its 3rd index and set them in a list;
2. Convert the list of dictionaries into an $\mathbf{X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the unigrams only. Each item in this list will be a dictionary. You will call it `X_cat`

In [ ]:
def get_data_points(dataset):
    return [data[3] for data in dataset]
X_cat = get_data_points(dataset_small_feat)

In [ ]:
X_cat[:2]

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

In [ ]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(X_cat)

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [ ]:
def get_lang_symbols(dataset):
    return [data[1] for data in dataset]
y_cat = get_lang_symbols(dataset_small_feat)

In [ ]:
y_cat[:5]

Extract the set of language symbols and build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2inx`and `inx2lang`.

In [ ]:
inx2lang = {}
lang2inx = {}
for index, lang in enumerate(set(y_cat)):
    inx2lang[index] = lang
    lang2inx[lang] = index

In [ ]:
inx2lang

In [ ]:
lang2inx

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [ ]:
y = [lang2inx[label] for label in y_cat]

In [ ]:
y[:5]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call you classifier `clf`

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=20, verbose=True)

In [ ]:
clf

### Training and Validation Sets

You will now split the dataset into training and a validation sets

#### We shuffle the indices

In [ ]:
indices = list(range(X.shape[0]))
np.random.shuffle(indices)
print(indices[:10])
X = X[indices, :]
y = np.array(y)[indices]

#### We split the dataset

In [ ]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [ ]:
clf.fit(X_train, y_train)

## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [ ]:
y_val_pred = clf.predict(X_val)

In [ ]:
print(y_val_pred[:10])
print(y_val[:10])

#### Evaluating

In [ ]:
accuracy_score(y_val, y_val_pred)

In [ ]:
y_symbols = lang2inx.keys()

In [ ]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

### Confusion Matrix

In [ ]:
confusion_matrix(y_val, y_val_pred)

Increase the number of iterations to improve the score. You may also change the parameters.

## Predict the language of a text

Now predict the languages of the strings below.

In [ ]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

Create features vectors from this list. Call this matrix `X_test`

In [ ]:
def get_test_features(dataset):
    ext_dataset = []
    for data in dataset:
        ext = count_chars(data)
        ext_dataset.append(ext)
    return ext_dataset
X_test_dicts = get_test_features(docs)

X_test = v.transform(X_test_dicts)

And run the prediction that you will store in a variable called `pred_languages`

In [ ]:
pred_languages = [inx2lang[s] for s in clf.predict(X_test).tolist()]

In [ ]:
pred_languages

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote last year for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.